In [2]:
# This code can be used for checking duplicated ids based on badge mac IDs
# Since badges use CRC16 to calcualte a default ID, these duplicates are possible

In [3]:
import crc16
import collections
import csv
import numpy as np

In [4]:
# Initial value for CRC16-CCITT is 0xffff
# See: https://github.com/willemwouters/NRF51/blob/master/sdk_6_1/nrf51822/Source/app_common/crc16.c 

def macArrayToCRC16(macArray):
    """
    macArray
    """
    tempArray = macArray[:]
    tempArray.reverse()
    key = struct.pack('%sB' % len(tempArray), *tempArray)
    crc = crc16.crc16xmodem(key,0xFFFF)
    return crc

def macStringToCRC16(macStr):
    # convert hex to bytes and reverse
    macstr = macStr.replace(':', '').decode('hex')[::-1]
    crc = crc16.crc16xmodem(macstr,0xFFFF)
    return crc


# basic tests
print(macStringToCRC16('CF:56:3F:E5:36:3F')) # 30837
print(macStringToCRC16('EF:21:18:08:F7:F6')) # 7384
print(macStringToCRC16('D2:92:7B:01:1A:A5')) # 9526

30837
7384
9526


In [5]:
macs_file = 'macs.csv'
with open(macs_file, 'rb') as f:
    reader = csv.reader(f)
    badge_macs = list(x[0] for x in reader)

In [6]:
mac_and_ID = [{'mac:':x,'id':macStringToCRC16(x)} for x in badge_macs]

In [7]:
ids = [d['id'] for d in mac_and_ID]
print (len(ids))

139


In [8]:
counter=collections.Counter(ids)
print(len(counter))

138


In [9]:
unique_ids = [x for x in counter]
duplicate_ids = []
for mid in unique_ids:
    if counter[mid] > 1:
        print("Duplicate: {}".format(mid))
        duplicate_ids.append(mid)
print(duplicate_ids)

Duplicate: 5084
[5084]


In [10]:
# print duplicates
for mid in duplicate_ids:
    for x in  mac_and_ID:
        if x['id'] == mid: print x


{'mac:': 'E7:06:BD:09:D5:3D', 'id': 5084}
{'mac:': 'C7:59:BE:8B:9C:E0', 'id': 5084}
